# Problem 4 : Estimation - BLP

In [26]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra,StatsFuns

Random.seed!(111);

In [34]:
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));
x = Array(df[!,[:x, :p]]);

Γ = ones(2,2)
Γ[1,2] = 0

δ = ones(size(df, 1));

df

Row,market,choice,shares,p,x,z1,z2,z3,z4,z5,z6
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,0.0099,0.000277852,-0.210973,-6.36336,-5.50945,-5.37102,-1.58364,-1.93707,-0.34181
2,1,2,0.2085,0.000457065,-1.55758,-5.19816,-4.39604,-4.26527,-2.86235,-3.49809,-2.33013
3,1,3,0.009,1.01684,-0.799984,-0.0408355,0.835758,-0.520122,-0.797271,0.768238,-0.973296
4,1,4,0.0348,0.109472,-1.33837,-1.53891,-1.1727,0.189448,-1.79817,1.51942,-0.264684
5,1,5,0.3661,5.38779,3.31633,0.758555,-0.70047,0.723864,-0.37094,-1.28846,0.0804491
6,1,6,0.0231,4.18812,2.15453,1.04464,-0.724283,-1.32533,-1.52045,-0.922602,1.71668
7,2,1,0.0078,0.000566978,0.470776,-5.87798,-4.01759,-5.99733,-1.18887,-1.8697,0.298796
8,2,2,0.0296,0.000313199,-0.153134,-6.24899,-4.75995,-5.71777,-1.21848,-3.64392,-1.61345
9,2,3,0.1604,0.724284,-2.18476,0.851501,0.426509,1.73536,-0.512458,-1.09451,-0.576743


In [ ]:
# simulate individual taste shocks from N(μ,Σ)
draw_sim = function(μ, Σ, N) # return N x L matrix
    # draw shocks
    ν = transpose(rand(MvNormal(μ, Σ), N))
    
    return ν
end

In [30]:
draw_sim([0;0], [1 0;0 1], 100)

100×2 transpose(::Matrix{Float64}) with eltype Float64:
 -2.22392    -1.68747
  0.497658   -0.587306
 -1.22048    -1.26904
  2.9424      1.84898
 -0.414499   -1.36067
 -1.40456    -0.305403
  0.370453   -0.93328
  0.165948    0.902509
  0.0452692  -1.94551
 -0.805055   -1.2845
  0.472479    1.47959
  0.0749692  -0.205567
  0.171761   -0.0647411
  ⋮          
 -0.145894   -0.693991
 -0.264349    0.251767
 -0.662192    0.974067
 -2.10149     0.64659
  0.124488    2.54002
 -1.0981     -0.876324
  1.51523     0.245813
 -1.10618     1.63682
  0.209489   -0.0493204
  1.19258    -0.436097
 -0.288937    0.869744
  1.06661     0.768633

# Part 1: BLP

## Inner loop
`get_shares` calculates the shares of each product in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of length $J$; and $v$ should be a vector of length $L$.

`delta_contraction` iterates the $\delta_{jt}$ in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of characteristics with length $J$; $s$ should be a vector of observed shares with length $J$; $v$ should be a vector of length $L$. 

`market_iterate` performs the contraction over each $t$ markets, it recoves $\delta_{jt}$, which is a vector of length $J \times T$.

In [ ]:
# get shares in a market given some fixed gamma and delta
get_shares = function(δ, Γ, x, v)
    # we want to get share_{jt} using simulated values of $v_i$ (drawn above)
    # shares should be vector of length J
    
    numerator = exp.(δ + x' * Γ * v)
    denominator = sum(numerator) + 1
    shares = sum(numerator ./ denominator) / size(v)[1]
    
    return shares
end

# inner loop: contraction to find δ
delta_contraction = function(δ₀, Γ, s, x, v, tol = 1e-12, max_iter = nothing)

    # here δ is a vector of length J
    δ = δ₀
    err = 1000
    n = 0
    
    while err < tol && maxed_iter
        δ_old = δ
        
        # update delta
        δ = δ_old + log.(s) - log.(get_shares(δ_old, Γ, x, v))
        
        # difference 
        err = maximum(abs.(δ - δ_old)) 
        
        # (optional) max iterations block
        n += 1
        if max_iter !== nothing
            maxed_iter = (n <= max_iter)
        end
    end
    
    return δ
end

# iterate over each market
market_iterate = function(initial, Γ, s_t, x_t, v, tol = 1e-12, max_iter = nothing)
   
    δ = []
    for t in 1:size(s_t)[1]
        s = s_t[t]
        x = x_t[t]
        δ₀ = ones(size(s)[1])
        push!(δ, delta_contraction(δ₀, Γ, s, x, v, tol, max_iter) ) 
    end
end

## Outer loop
`residuals` does IV-GMM using the provided weighting matrix. z_jt should be a matrix of $Z$ excluded and included intruments of size $TJ \times Z$. Returns linear parameters (vector of length $2$) and $\xi_{jt}$ residuals (vector of length $J \times T$)

`gmm_objective` Reads in $TJ$-length vector $x$_jt and $TJ \times Z$ matrix $z$_jt. Calculates sample moments (size of instrument vector, $Z$) and optimal weighting matrix ($Z \times Z$). Returns scalar objective and matrix.

In [ ]:
# returns residuals for a given δ, estimates linear parameters given instruments
residuals = function(δ_jt, x_jt, z_jt, W = nothing)
    # if weighting matrix not supplied
    if W === nothing
        w = inv(z_jt' * z_t)
    end
    
    # iv-gmm
    θ₁ = inv(x_jt' * z_jt * W * z_jt' * x_jt) * (x_jt' * z_jt * W * z_jt' * δ_jt)
    ξ_jt = δ_jt - x_jt * θ_1
    
    return ξ_jt, θ₁ 
    
end

# calculates gmm objective for outer loop
gmm_objective = function(ξ_jt, z_jt)   
    # empirical moments, weighting matrix
    g = (ξ_jt' * z_jt) / size(ξ_jt)[1] 
    W = inv((z_jt .* ξ_jt.^2 )' * z_jt) * size(ξ_jt)[1]
    
    # gmm objective
    G = g * W * g'
    
    return G, W
end

# performs outer loop, keeps track of weighting matrix
outer_loop = function!(θ₂, W = nothing, s_t, x_t, z_jt, v, tol = 1e-12, max_iter = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    # Perform inner loop
    δ = market_iterate(initial, Γ, initial, Γ, s_t, x_t, v, tol = 1e-12, max_iter = nothing) # what is initial??
    
    # convert to JT x 1 (stacked J x 1 vectors for each t)
    δ_jt = vec(reduce(hcat,δ)') 
    x_jt = vec(x_t')
    
    # intermediate step
    ξ_jt, θ₁ = residuals(δ_jt, x_jt, z_jt, W)
    
    # gmm step
    G, W = gmm_objective(ξ_jt, z_jt)
    
    return G
end

In [47]:
summed_jt_i = exp.(δ .+ x * Γ * ν') # this returns a 600 (jt) by 20 (i) 
sum(summed_jt_i, dims=2) # this sums across individuals 

600×1 Matrix{Float64}:
     57.00582229965489
    233.16347456628938
     62.23148664334593
    139.0192307176588
      1.9572010631035707e13
      7.5191040125367e9
     64.61729047613422
     55.921108756838976
    176.30447614723312
   4600.2410653501875
      1.1154570965762811e6
      4.802172137882022e8
     54.97046030264811
      ⋮
     92.05096050484761
    183.74617118856838
    707.4496820917077
  87825.27102073307
   3984.880690599852
   6384.160433719341
     54.68272891024609
     57.629424848527094
  89979.25568635199
 175360.58854399397
 112097.32688149536
      3.2869182094897693e19

In [3]:
sum(summed_jt_i, dims=2) # this should sum across goods/markets

LoadError: UndefVarError: summed_jt_i not defined

In [108]:
z = []
push!(z,[1, 5, 1, 2])
push!(z,[2, 3, 0, 4])
push!(z,[7, 9, 5, 6])

vec(reduce(hcat,z)')

12-element reshape(adjoint(::Matrix{Int64}), 12) with eltype Int64:
 1
 2
 7
 5
 3
 9
 1
 0
 5
 2
 4
 6